# Second Iteration of Simulation SAoI

In this notebook, we will perform the second iteration of the SAoI simulation, with the goal of determining the optimal signal transmission frequency to minimize SAoI in the M/M/1 queue with a varying Carbon Intensity (CI) index.

In [1]:
import numpy as np 
from numpy import random 
import matplotlib.pyplot as plt 
from math import sin, pi
from collections import deque #We use deque for simulation of a queue

# We need to define function sinus that will represent value of CI over the time. 
def CI(x):
    """This function simulates value of CI over the time.
    """
    return 100 * sin(x/(2*pi*3600*24)) + 300

In [2]:
def create_arrivals(arrival_rate, total_time):
    """Returns the list of arrival times of a M/M/1 queue
    with arrival rate arrival_rate and total time of the
    simulation total_time

    Parameters
    ----------
    arrival_rate: float
        Number between 0 and 1 that represents arrival rate of designed 
        M/M/1 queue

    total_time: int
        Total time of the simulation

    Returns
    --------
    list
       list of arrivals
    """ 
    list_of_arrivals = [0]
    for i in range(2*int(total_time/arrival_rate)):
        t = list_of_arrivals[-1] + random.exponential(1/arrival_rate)
        list_of_arrivals.append(t)
    return list_of_arrivals

In [3]:
#we create class Packet in which we have objects that are our update packets
class Packet:
    def __init__(self, size, trans, gen_time):
        self.size = size
        self.trans = trans
        self.gen_time = gen_time

    def transmit(self, time):
        self.trans -= time

In [7]:
def simulacija(ar, tt=36000*24, ts=0.05):
    """Returns the average CT (carbon footprint) and AoI (age of 
    information) among time tt with time steps ts for M/M/1 queue 
    with arrival rate ar and serving rate 1.
    
    Parameters
    ----------
    ar: float
        Number between 0 and 1 that represents arrival rate of 
        designed M/M/1 queue.
        
    tt: int
        Total time of the simulation.

    ts: float
        Number that represents the time step that we take at each 
        iteration of the loop.

    Returns
    --------
    float list
        Numbers that represent the average CT and AoI.
    """
    #we need list in which all of the times of arrivals are stored. 
    #Times in this list are absolute and not interarrival.
    list_of_arrivals = create_arrivals(ar, tt)

    #we need list in which all of the times of servings are stored.
    #times in this list are interserving times and not absolute times.
    list_of_servings = random.exponential(scale=1, size=2*int(tt))

    ar_ind = 0 #helps us take the element with right index out of list_of_arrivals.
    ser_ind = 0 #helps us take the element with right index out of list_of_servings.
    time = 0 
    age_list = [] #here current AoI is stored at every time step
    CT_list = [] #here current CT is stored at every time step  
    age = 0
    queue = deque([]) 
    packet_list = []
    inter_serving_time = 0 
    for i in range(int(tt/100)):
        #we use this double loop so that we refresh energy to 0 after every 100 time units. 
        for j in range(int(100/ts)):
            energy = 0
            time += ts
            age += ts
            inter_serving_time += ts 
            if time > list_of_arrivals[ar_ind]:
                energy += 18
                packet_list.append(Packet(6, 5, time))
                ar_ind += 1
            for inx in range(len(packet_list)):
                pack = packet_list[inx]
                pack.transmit(ts)
                packet_list[inx] = pack
            if len(packet_list) != 0:
                if packet_list[0].trans < 0:
                    queue.append(packet_list[0])
                    packet_list = packet_list[1:]
            if inter_serving_time > list_of_servings[ser_ind] and len(queue) != 0:
                inter_serving_time = 0
                energy += list_of_servings[ser_ind - 1] * 18
                age = time - queue[0].gen_time
                energy += age - 5
                queue.popleft()
                ser_ind += 1
            age_list.append(age)
        CT = energy * CI(time)
        CT_list.append(CT)
    #note that CT_list has 100/ts times less elements as age_list. When we take the average of both it is not the problem anymore and we can compare them.
    CT_avg = sum(CT_list)/len(CT_list)
    age_avg = sum(age_list)/len(age_list)
    return [CT_avg, age_avg]

    simulacija(0.5, tt=100000, ts=0.05)   